In [1]:
import time
import itertools
import json
import panel as pn
import numpy as np
import pandas as pd
import holoviews as hv
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from IPython.display import HTML , IFrame, YouTubeVideo, clear_output

from panel.widgets import Tqdm

from folium.plugins import HeatMap, HeatMapWithTime, MarkerCluster

from bokeh import models, plotting, io
from bokeh.layouts import column
from bokeh.sampledata import stocks
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, RangeTool, BoxAnnotation

from holoviews import opts
from holoviews.streams import Pipe, Buffer
from holoviews.operation.timeseries import rolling, rolling_outlier_std

from scipy.linalg import LinAlgError

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

hv.extension('bokeh')
hv.renderer('bokeh').theme= 'dark_minimal'

pd.options.display.max_rows = 10

%matplotlib inline

KeyboardInterrupt: 

* Youtube Channel4 Video :*[About Sharing](#lower)

In [ ]:
YouTubeVideo("vFZZF39fgWM", width=1200, height=500, allow_autoplay=False, controls=1, loop=1, modestbranding=0, rel=0)

<p style= "background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:400%;text-align:center;border-radius:10px 10px;border-style:solid;border-width:3px;border-color:#000000;"><b> Daily Cases by Country and Continent (COVID-19 variant EDA)</b></p>

# Please give me an UPVOTE if you can. Your UPVOTE will be a great encouragement to me!

<a id='top'></a>


- [懸念される変異株（VOC）variant of concern (VOC)](https://www.who.int/en/activities/tracking-SARS-CoV-2-variants/) : Warld Helth Organization : Tracking SARS-CoV-2 variants
- [COVID-19 Vaccinations in the United States](https://covid.cdc.gov/covid-data-tracker/#vaccinations_vacc-total-admin-rate-total) : Centers for Disease Control and Prevention アメリカ疾病予防管理センター CDC
- [GOV.UK Coronavirus (COVID-19) in the UK](https://coronavirus.data.gov.uk/) : Government

- 新しいオミクロンCOVID-19バリアントの進行を追跡する

<p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Data Fields</p>


* location-これは、バリアント情報が提供される国です。
* date-データ入力の日付。
* variant-これは、このデータエントリに対応するバリアントです。
* num_sequences-処理されたシーケンスの数（国、バリアント、および日付）。
* perc_sequences-シーケンスの総数（国、バリアント、および日付）からのシーケンスのパーセンテージ。
* num sequence total-シーケンスの総数（国、バリアント、および日付）。

<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:200%;text-align:center;border-radius:10px 10px;">CONTENTS</p>

* [1. Data Pre-Processing & Feature Engineering](#1)
* [2. EDA Visualization : HoloViews Bokeh Panel Folium](#2)
* [3. Plotly Animation](#3)
* [4. Create lat lon list](#4)
* [5. Folium HeatMap](#5)
* [6. Coropleth Map : variant](#6)
    
    
</div>

<a id="1"></a>
### <p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:250%;text-align:center;border-radius:10px 10px;">Data Pre-Processing & Feature Engineering</p>

* コロプレスマップ用にJsonをロード
* Jsonデータの整合性チェックを行い、値を事前に変換しておく。
* 各国の緯度経度をデータセットに結合する

In [ ]:
wc_v = r'D:\JupyterNotebook\World_Countries_json\world-countries.json'

In [ ]:
train_data = pd.read_csv('D:\JupyterNotebook\Covid_Omicron\covid-variants.csv')

In [ ]:
v_test = train_data['variant'].unique()
v_test, len(v_test)

In [ ]:
train_data.info()

In [ ]:
train_data.describe().style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.rename(columns={'location': 'Country', 'date': 'Date',
                           'num_sequences_total': 'Total', 'perc_sequences': 'seq_percentile'}, inplace=True)

train_data['Dates'] = pd.to_datetime(train_data['Date'])
train_data['Year'] = train_data['Dates'].dt.year
train_data['Month'] = train_data['Dates'].dt.month

lat_lon_1 = pd.read_csv('D:\JupyterNotebook\Country_latitude_longitude\\world_country_and_usa_states_latitude_and_longitude_values.csv')
lat_lon_1.replace('United States', 'United States of America', inplace=True)
train_data.replace('United States', 'United States of America', inplace=True)

lt_data_1 = lat_lon_1.loc[:, ['country_code', 'country', 'latitude', 'longitude']].rename(columns={'country': 'Country', 'latitude': 'lat', 'longitude': 'lon'})
train_data = pd.merge(lt_data_1, train_data, how='inner', on='Country')

In [ ]:
year_avg = pd.pivot_table(train_data, values=['num_sequences'], index='Year', aggfunc='sum')
group_year_avg = train_data.groupby(['Country', 'Year']).mean()

In [ ]:
asia_continent = ['Afghanistan', 'Bahrain', 'Bangladesh', 'Cambodia', 'China','Hong Kong, China', 'Brunei', 'Qatar',
                  'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan', 'South Korea',
                  'Korea, Rep.', 'Kuwait', 'Lebanon', 'Malaysia', 'Mongolia', 'Myanmar', 'Nepal', 'Oman', 'Russia',
                  'Pakistan', 'Philippines', 'United Arab Emirates', 'Singapore', 'Sri Lanka', 'Syria', 'Taiwan',
                  'Thailand', 'Vietnam', 'West Bank and Gaza', 'Yemen, Rep.', 'Maldives', 'Kazakhstan', 'Hong Kong']

europe_continent = ['Albania', 'Austria', 'Belgium', 'Bosnia and Herzegovina','Bulgaria', 'Croatia',
                    'Czech Republic', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Hungary',
                    'Iceland', 'Ireland', 'Italy', 'Montenegro', 'Netherlands', 'Norway', 'Poland',
                    'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Lithuania',
                    'Switzerland', 'Turkey', 'United Kingdom', 'Moldova', 'Malta', 'Luxembourg', 'Liechtenstein', 
                    'Latvia', 'Kosovo', 'Georgia', 'Cyprus', 'Estonia', 'Ukraine', 'Monaco']

africa_continent = ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon',
                    'Central African Republic', 'Chad', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.',
                    "Cote d'Ivoire", 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Ethiopia', 
                    'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Seychelles',
                    'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius',
                    'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Reunion', 'Rwanda',
                    'Sao Tome and Principe', 'Senegal', 'Sierra Leone', 'Somalia', 'South Africa',
                    'Sudan','Swaziland', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe']

americas_continent = ['Argentina', 'Bolivia', 'Brazil', 'Canada', 'Chile', 'Colombia', 'Costa Rica',
                      'Cuba', 'Dominican Republic', 'Ecuador', 'El Salvador', 'Guatemala', 'Haiti',
                      'Honduras', 'Jamaica','Mexico', 'Nicaragua', 'Panama', 'Paraguay', 'Peru',
                      'Puerto Rico', 'Trinidad and Tobago', 'United States of America', 'Uruguay', 'Venezuela',
                      'Belize', 'Aruba', 'Suriname']

oceania_continent = ['Australia', 'New Zealand', 'Papua New Guinea', 'Fiji']

In [ ]:
continent_dict = {'Asia': asia_continent, 
                  'Europe': europe_continent, 
                  'Africa': africa_continent, 
                  'Americas': americas_continent, 
                  'Oceania': oceania_continent}
cont_dict = {val:key for key, lst in continent_dict.items() for val in lst}
train_data['continent'] = train_data['Country'].map(cont_dict)

In [ ]:
train_data.loc[train_data['continent'].isnull()]

In [ ]:
train_data.head(10).style.background_gradient(cmap='mako_r', text_color_threshold=0.02)

<a id="2"></a>
### <p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:250%;text-align:center;border-radius:10px 10px;">EDA Visualization : HoloViews Bokeh Panel Folium</p>

<p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Year : Country</p>

In [ ]:
def year_total_plot(data, year):
    data_g = data.reset_index()
    vdims = [('Total', 'TOTAL')]
    ds = hv.Dataset(data_g, ['Year', 'Country'], vdims)

    states = list(data_g['Country'])

    bars = ds.select(State=states, Year=year).to(hv.Bars, ['Year', 'Country'], 'Total').sort()
    bars.opts(
        opts.Bars(width=1200, height=500, tools=['hover'], xrotation=90, show_legend=False, title='Data Year : {}'.format(year)))
    return bars

In [ ]:
year_total_plot(group_year_avg, (2020))

In [ ]:
year_total_plot(group_year_avg, (2021))

<p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Year : variant</p>

In [ ]:
group_year_avg = train_data.groupby(['Year', 'variant']).mean()

In [ ]:
def variant_plot(data, year):
    data_g = data.reset_index()
    vdims = [('Total', 'TOTAL')]
    ds = hv.Dataset(data_g, ['Year', 'variant'], vdims)

    states = list(data_g['variant'])

    bars = ds.select(State=states, Year=year).to(hv.Bars, ['Year', 'variant'], 'seq_percentile').sort()
    bars.opts(
        opts.Bars(width=1200, height=500, tools=['hover'], xrotation=90, show_legend=False, title='Data Year : {}'.format(year)))
    return bars

In [ ]:
variant_plot(group_year_avg, (2020))

In [ ]:
variant_plot(group_year_avg, (2021))

In [ ]:
variant_plot(group_year_avg, (2022))

<p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Dates : num_sequences</p>

In [ ]:
def date_num_plot(data):
    date_avg = pd.pivot_table(data, values='Total', index='Date', aggfunc='mean')
    bar_plot = hv.Bars(date_avg)
    lin_plot = hv.Curve(date_avg)
    vline = hv.VLine(pd.datetime(2021, 1, 4))
    bar_lin = (bar_plot * lin_plot * vline)
    bar_lin.opts(opts.Bars(width=1200, alpha=0.8, line_width=2, show_grid=True, xrotation=90, stacked=True, yformatter="%.0f",
                           color=hv.Cycle('Category20'), title='Covid-19', toolbar='above', tools=['hover']),
                 opts.VLine(color='red', line_width=100))
    return bar_lin

In [ ]:
date_num_plot(train_data)

<p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Total : variant</p>

In [ ]:
def total_variant(data):
    variant = data.groupby(['Date', 'variant'])['Total'].sum().reset_index()
    table = hv.Table(variant).to.bars(['Date', 'variant'], 'Total', [])
    table.opts(opts.Bars(color=hv.Cycle('Category20'), show_legend=False, stacked=True, yformatter="%.0f",
                         tools=['hover'], width=1000, xrotation=90)) #  height=700,
    return table

In [ ]:
total_variant(train_data)

<a id="3"></a>
### <p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:350%;text-align:center;border-radius:10px 10px;">Plotly Animation</p>

* 結合した大陸情報に沿ってアニメーションによる可視化を行う。

In [ ]:
px.scatter(train_data, x="num_sequences", y="Total", animation_frame="Date", animation_group="Country",
           size="Total", color="continent", hover_name="Country", range_y=[-5000, 200000], 
           range_x=[-2000, 60000], size_max=60, template="plotly_dark")

In [ ]:
px.bar(train_data, x="variant", y="seq_percentile", color="variant", template="plotly_dark",
       animation_frame="Date", range_y=[0, 5000])

[最新のものは主にデルタバリアントでした。ここ数週間、B.1.1.529バリアントの検出と一致して、感染が急増しています。最初に確認されたB.1.1.529感染は、2021年11月9日に収集された検体からのものでした。](https://www.who.int/news/item/26-11-2021-classification-of-omicron-(b.1.1.529)-sars-cov-2-variant-of-concern) : Warld Helth Organization ,Classification of Omicron (B.1.1.529): SARS-CoV-2 Variant of Concern (VOC)
26 November 2021 

* アニメーション可視化でも見られるようにDelta株から一時的に other, non_whoが発生し、翌からOmicronが徐々に増幅している事が見て取れる。 

<a id="4"></a>
### <p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:250%;text-align:center;border-radius:10px 10px;">Create lat lon list </p>

In [ ]:
pd.options.display.max_rows = 10

In [ ]:
tqdm_outer = Tqdm()

In [ ]:
def variance_lat_lon(train_data, query="variant=='Omicron' & num_sequences!=0"):
    test_data_1 = train_data.query(query)
    lat = []
    lon = []
    for i in tqdm_outer(range(0, len(test_data_1))):
        test_num = test_data_1['num_sequences'].iloc[i]
        lat_a, lon_c = test_data_1.loc[:, ['lat', 'lon']].iloc[i]
        for _ in itertools.repeat(None, test_num):
            lat.append(lat_a)
            lon.append(lon_c)
    return pd.DataFrame(data={'lat': lat, 'lon': lon},columns=['lat', 'lon'])

In [ ]:
pn.Column('#Latitude_Longitude' ,tqdm_outer , background='WhiteSmoke')

In [ ]:
lat_lon_data = variance_lat_lon(train_data, "variant=='Omicron' & num_sequences!=0")

<a id="5"></a>
### <p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:250%;text-align:center;border-radius:10px 10px;">Folium HeatMap </p>

In [ ]:
def heat_map_folium(data):
    geo_list = data.values.tolist()
    m = folium.Map(location=[0, 0], zoom_start=2, tiles='CartoDB dark_matter')
    HeatMap(geo_list).add_to(m)
    return m

<p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Omicron Sequences HeatMap</p>

- ヒートマップの位置は国の緯度経度を基準にしているため、国の中心付近が濃くなりますが、これはその地域の感染数が極端に多いということではありません。表示サイズはデフォルト設定です。

In [ ]:
heat_map_folium(lat_lon_data)

<a id="6"></a>
### <p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:350%;text-align:center;border-radius:10px 10px;">Coropleth Map : variant</p>

In [ ]:
wc_v = r'D:\JupyterNotebook\World_Countries_json\world-countries.json'

In [ ]:
def variant_coropleth(data, variant='Delta'):
    query_data = data.query("variant=='{}'".format(variant)).groupby('Country').sum().reset_index()
    c_m_data = query_data[['Country', 'num_sequences']]

    chor = folium.Map(location=[0, 0], zoom_start=2, tiles='stamenterrain')
    tiles = ['stamenwatercolor', 'openstreetmap', 'cartodbpositron', 'stamenterrain']
    for tile in tiles:
        folium.TileLayer(tile).add_to(chor)
    f_chor = folium.Choropleth(geo_data=wc_v,
                               data=c_m_data,
                               columns=['Country', 'num_sequences'],
                               key_on='feature.properties.name',
                               fill_color='Set1', fill_opacity=0.6, line_opacity=0.6, 
                               highlight=True, legend_name='Covid: {} Cases'.format(variant)).add_to(chor)
    f_chor.geojson.add_child(folium.features.GeoJsonTooltip(['name'], labels=False))

    folium.LayerControl().add_to(chor);
    #chor.save("Covid_case_Omicron.html")
    return chor

- 右上のレイヤーコントロールを使用して地図のTile表示を切り替え可能。

- 関数の第2引数にバリアントを入力することで、表示を切り替えることができます。デフォルトではオミクロンに設定されています。

- グレーの位置はデータの存在しない地域です。

In [ ]:
train_data['variant'].unique()

In [ ]:
variant_coropleth(train_data, 'Omicron')

In [ ]:
variant_coropleth(train_data, 'Delta')

In [ ]:
variant_coropleth(train_data, 'Alpha')

<p style="background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">CDC Covid Data Tracker</p>

In [ ]:
html_cdc = """<a href="https://covid.cdc.gov/covid-data-tracker/#datatracker-home?c_cid=CDT_Button" 
                title="image of SARS-CoV-2"><img src="http://www.cdc.gov/coronavirus/2019-ncov/images/COVID_Data_Tracker_Widget_card.png"
                style="width:300px; height:178px; border:0px;" alt="image of Coronavirus Disease 2019 (COVID-19)"/></a>"""
HTML(html_cdc)

---
<a id="lower"></a>
* Japan:[経済産業省(Ministry of Economy, Trade and Industry):電子商取引及び情報財取引等に関する準則](https://www.meti.go.jp/press/2020/08/20200828001/20200828001.html)
 - リンクを張ること自体により、公衆送信、複製のいずれも行われるわけではないから、複製権侵害、公衆送信権侵害のいずれも問題にならないものと考えられる。
 - サーフェスリンク、ディープリンク、イメージリンク、フレームリンク、インラインリンクの個別の態様でのリンクを張る行為自体においては、原則として著作権侵害の問題は生じないと考えるのが合理的である。
